# Interpolace
Vykreslení výsledků z genetického algoritmu. Pro různé běhy si ukládáme, v jaké generaci a o kolik se zvýšila fitness hodnota. Máme celkem 20 běhů s různými generacemi, kde došlo ke změně.

## Načítání dat
Načteme data ze souborů v archivu [runs.zip](runs.zip). Každý soubor má následující formát:
```
gen=0,fit=0.00
gen=3,fit=0.04
gen=7,fit=0.04
gen=20,fit=0.09
gen=21,fit=0.33
gen=27,fit=0.49
gen=35,fit=0.53
gen=89,fit=0.68
gen=90,fit=0.74
```

Pro dekódování použijeme 2x zanořenou funkci `map`, která ve vyšší úrovni rozdělí řádky podle čárek a v nižší podle '='. Tento soubor potom převede rovnou na následující zápis
```
[
    {("gen", "0"), ("fit", "0.00")},
    {("gen", "3"), ("fit", "0.04")},
    {("gen", "7"), ("fit", "0.04")},
    {("gen", "20"), ("fit", "0.09")},
    {("gen", "21"), ("fit", "0.33")},
    {("gen", "27"), ("fit", "0.49")},
    {("gen", "35"), ("fit", "0.53")},
    {("gen", "89"), ("fit", "0.68")},
    {("gen", "90"), ("fit", "0.74")}
]
```

Potom se vytvoří `pd.DataFrame`, sloupce se hromadně konvertují a přidá se informace o běhu. Všechny tyto postupy byste měli znát z předcházejících přednášek


In [ ]:
###
from zipfile import ZipFile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# načtení dat (viz popis nahoře)
alld = []
with ZipFile("runs.zip") as zf:
    for fn in zf.namelist():
        with zf.open(fn) as f:
            # dekodovani gen=X,fit=XX
            df_loc = pd.DataFrame(list(map(
                lambda x: dict(map(lambda y: tuple(y.split("=")),
                                   x.decode().strip().split(","))),
                f.readlines()
            )))
            df_loc["gen"] = df_loc["gen"].astype("i")
            df_loc["fit"] = df_loc["fit"].astype("f")
            df_loc["run"] = fn
            alld.append(df_loc)

df = pd.concat(alld, ignore_index=True)
df

In [ ]:
###
# Jak data vypadají?
sns.lineplot(data=df, x="gen", y="fit", hue="run", marker="o", legend=False)

## Konvergence
Nyní chceme určit tzv. konvergenci - tj. jak se mění fitness v průběhu generací. Nejlépe by bylo zobrazit i _confidence interval_ pro určení 95% rozsahu.

In [ ]:
# Zobrazit konvergenční křivku pro dataframe df
sns.lineplot(data=df, x="gen", y="fit", legend=False)

V grafu ale vznikly škaredé skoky. Ty jsou způsobené tím, že počet záznamů pro každou generaci je rozdílný (měl by být 20, což je počet skoků).

In [ ]:
###
dfv = df.groupby("gen").count().reset_index()
plt.bar(dfv["gen"] + 0.5, dfv["run"])
_= plt.gca().set(xticks=np.linspace(0, 100, num=5), xticklabels=np.linspace(0, 100, num=5), 
          ylabel="# runs", xlabel="gen")

## Interpolace pro všechny generace
Proto musíme vytvořit nový DataFrame, který pro všechny generace má uvedenou fitness.

In [ ]:
from scipy.interpolate import interp1d ###
# Vytvoříme si nový prostor X - který může být např: 
#  - celý prostorn 0 - 100
#  - unikátní hodnoty gen v datasetu df
#  - unikátní hodnoty gen a hodnoty gen - delta v df
#  - ...
X = np.linspace(0, 100, 101) ###

# pro všechny běhy vytvoříme nový DataFrame se správnými hodnotami fitness 
# použijeme 1D interpolaci i s extrapolací tak, že interpolujeme předchozí hodnotou
alld = [] ###
for r, d in df.groupby("run"):
    f = interp1d(d["gen"], d["fit"], kind="previous", fill_value="extrapolate")
    df_loc = pd.DataFrame({"gen": X, "fit": f(X)})
    df_loc["run"] = r
    alld.append(df_loc)
df2 = pd.concat(alld) ###
df2 ###

Nyní se podíváme na to, jak celý nový dataset vypadá. Všimneme si toho, jak jsou tvořeny schody a že funkce jsou nyní definované až do konce

In [ ]:
# Vykreslení čárového grafu z df2 a přes něj vykreslený scatterplot starého dataframe df
ax = sns.lineplot(data=df2, x="gen", y="fit", hue="run", legend=False)
sns.scatterplot(data=df, x="gen", y="fit") # puvodni hodnoty

## Vykreslení konvergenční křivky
Nyní můžeme vykreslit celou konvergenční křivku, nechat z ní spočítat průměr a confidence interval. Jsou samozřejmě možné další modifikace - jako např. zobrazit medián.

In [ ]:
# Vykreslení konvergenční křivky
sns.lineplot(data=df2, x="gen", y="fit", legend=False)


In [ ]:
###
# Vykreslení mediánů starých dat a prokládaných dat.
ax = sns.lineplot(data=df2, x="gen", y="fit", legend=False, estimator=np.median, ci=None, label="Medián fitness s proložením")
sns.lineplot(data=df, x="gen", y="fit", legend=False, ax=ax, estimator=np.median, ci=None, label="Medián fitness bez proložení")
plt.legend()

Hotovo!